<a href="https://colab.research.google.com/github/skalthafhussain/althaf/blob/main/HOUSE_PRICE_PREDICTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

HOUSE PRICE PRIDICTION

In [2]:
# Generic Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Dataset
from sklearn.datasets import fetch_california_housing


In [3]:
# get the data set

data = fetch_california_housing()

In [4]:
print (data.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

In [5]:
#independent data
df = pd.DataFrame(data=data.data,  columns=data.feature_names)
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [6]:
# dependent data
df['target']= data.target
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


EDA

In [7]:
!pip install sweetviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 53.0 MB/s eta 0:00:00


In [8]:
import sweetviz as sv

report = sv.analyze(df)
report.show_html("./report.html")

                                             |          | [  0%]   00:00 -> (? left)

Report ./report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


DATA PRE-PROCESSING

In [9]:
## Feature Engineering

from geopy.geocoders import Nominatim
import time
# use a unique and descriptive user agent
geolocator = Nominatim(user_agent= 'my_geocoding_app')
# Introduce a delay between requests to avoid rate limiting
time.sleep(2)



location = geolocator.reverse("37.88"+" , "+"-122.23").raw # Remove .raw["address"]

print(location)
if location:
    address = location["address"]# Access "address" directly from the location dictionary

    print(address)
else:
    print("Unable to retrieve address. Check your coordinates and rate limits.")








{'place_id': 282928603, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'way', 'osm_id': 665384454, 'lat': '37.87563745', 'lon': '-122.22856355341203', 'class': 'leisure', 'type': 'nature_reserve', 'place_rank': 30, 'importance': 9.99999999995449e-06, 'addresstype': 'leisure', 'name': 'Ecological Study Area', 'display_name': 'Ecological Study Area, Centennial Drive, La Loma Park, Oakland, Alameda County, California, 94720, United States', 'address': {'leisure': 'Ecological Study Area', 'road': 'Centennial Drive', 'neighbourhood': 'La Loma Park', 'city': 'Oakland', 'county': 'Alameda County', 'state': 'California', 'ISO3166-2-lvl4': 'US-CA', 'postcode': '94720', 'country': 'United States', 'country_code': 'us'}, 'boundingbox': ['37.8675298', '37.8837316', '-122.2483939', '-122.2232091']}
{'leisure': 'Ecological Study Area', 'road': 'Centennial Drive', 'neighbourhood': 'La Loma Park', 'city': 'Oakland', 'county': 'Alameda County', 'state': 

In [10]:
def location (cord): #The location function is now defined within the same scope as the loop

  latitude = str(cord[0])
  longitude = str(cord[1])
  location = geolocator.reverse( latitude+","+longitude).raw["address"] # raw returns a dictionary
  # if the values are missing replace by a empty string

  if location.get("Road") is None:
    location["Road"] = None

  if location.get("Country") is None:
    location["Country"] = None

  loc_update["Country"].append(location["Country"] )
  loc_update["Road"].append(location["Road"] )

In [11]:
"""
import pandas as pd
import pickle
from geopy.geocoders import Nominatim
from sklearn.datasets import fetch_california_housing
import time


# Use a more descriptive User-Agent

geolocator = Nominatim(user_agent='my-geocoding-app/1.0',timeout=10)

# Load or create your dataframe here
#  if you have a  data set fetch_california_housing:
housing= fetch_california_housing(as_frame=True) #data set ,as_frame will give a pandas dataframe

# extract the data into a DataFrame
df=housing.frame #Correct way to get the DataFrame from fetch_california_housing

df['target'] =housing.target# Use housing.target to access the target values



loc_update = {"Country":[],
              "Road":[],
              "neighbourhood":[],

           }
# Define the location function before it is called in the loop
def location (cord):

  latitude = str(cord[0])
  longitude = str(cord[1])
  location = geolocator.reverse( latitude+","+longitude).raw["address"] # raw returns a dictionary
  # if the values are missing replace by a empty string

  if location.get("Road") is None:
    location["Road"] = None

  if location.get("Country") is None:
    location["Country"] = None

  loc_update["Country"].append(location["Country"] )
  loc_update["Road"].append(location["Road"] )


for i,cord in enumerate(df.iloc[:, 6 :-1].values):

  location(cord)
  # continiously Reading our data and saving it on the go !!
  pickle.dump(loc_update, open("loc_update.pickle","wb")) # Use 'wb' for writing binary

  if i %100 == 0:
    print(i)
    time.sleep(1)  # Introduce a 1-second delay between every 100 requests
    """

'\nimport pandas as pd\nimport pickle\nfrom geopy.geocoders import Nominatim\nfrom sklearn.datasets import fetch_california_housing\nimport time\n\n\n# Use a more descriptive User-Agent\n\ngeolocator = Nominatim(user_agent=\'my-geocoding-app/1.0\',timeout=10)\n\n# Load or create your dataframe here\n#  if you have a  data set fetch_california_housing:\nhousing= fetch_california_housing(as_frame=True) #data set ,as_frame will give a pandas dataframe\n\n# extract the data into a DataFrame\ndf=housing.frame #Correct way to get the DataFrame from fetch_california_housing\n\ndf[\'target\'] =housing.target# Use housing.target to access the target values\n\n\n\nloc_update = {"Country":[],\n              "Road":[],\n              "neighbourhood":[],\n\n           }\n# Define the location function before it is called in the loop\ndef location (cord):\n\n  latitude = str(cord[0])\n  longitude = str(cord[1])\n  location = geolocator.reverse( latitude+","+longitude).raw["address"] # raw returns a 

In [12]:
import pickle
import os
import pandas as pd  # Import pandas with the alias 'pd'



# Get the current working directory
current_directory = os.getcwd()
print("Current working directory:", current_directory)  # Print the current directory for verification


# Construct the full path to the pickle file
file_path = os.path.join(current_directory, "loc_update.pickle")

# Try to load the pickle file
try:
   with open(file_path, "rb") as file:
        loc_update = pickle.load(file) # load loc_update from the pickle file
        print("Pickle file loaded successfully.")
        print(loc_update)
except FileNotFoundError:
    print("Pickle file not found. Make sure it has been created by running the previous cells and is in the current working directory.")
    loc_update = {"Country":[], "Road":[], "neighbourhood":[]} # Initialize loc_update if the file is not found

except Exception as e:
    print("An error occurred while loading the pickle file:", e)
    loc_update = {"Country":[], "Road":[], "neighbourhood":[]} # Initialize loc_update if an error occurs
    # Check lengths and fix if necessary
lengths = [len(v) for v in loc_update.values()]
min_length = min(lengths)
for key in loc_update:
    loc_update[key] = loc_update[key][:min_length] # Trim lists to the shortest length




     # Assuming 'location' is a dictionary, these lines should likely be within a function or loop
# For example, if you want to append these values after loading the pickle, you could do:
for key in ["Country", "Road", "neighbourhood"]:
  #  loc_update[key].append(location.get(key))

    for i,cord in enumerate(df.iloc[:, 6 :-1].values):

        if i % 100 == 0: # Indent this block under the if statement
           pickle.dump(loc_update, open("loc_update.pickle","wb")) # Use 'wb' for writing binary
           print(i)
           time.sleep(1)  # Introduce a 1-second delay between every 100 requests




  # Now you can use loc_update to create the DataFrame

loc = pd.DataFrame(loc_update)


loc.info()



Current working directory: /content
Pickle file not found. Make sure it has been created by running the previous cells and is in the current working directory.
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
1580

In [13]:
import pandas as pd  # Import pandas with the alias 'pd'

# If you don't have loc_update already, initialize it:
loc_update = {"Country": [], "Road": [], "neighbourhood": []}
# Load or create your dataframe here
#  if you have a  data set fetch_california_housing:
from sklearn.datasets import fetch_california_housing
housing= fetch_california_housing(as_frame=True) #data set ,as_frame will give a pandas dataframe

# extract the data into a DataFrame
df=housing.frame #Correct way to get the DataFrame from fetch_california_housing

df['target'] =housing.target# Use housing.target to access the target values





# add the new features to our dataframe

for i in loc_update.keys():
   # Create a Series with the same index as the DataFrame, filling missing values with None
  df[i] = pd.Series(loc_update[i], index=df.index[:len(loc_update[i])]).reindex(df.index)

  df = df.sample(axis=0, frac=1)
  df.head(10)

In [14]:
## drop latitude, longitude and the neighbourhood columns
df.drop(["Latitude","Longitude","neighbourhood"],axis=1,)
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal,target,Country,Road,neighbourhood
1055,3.0905,12.0,5.122490,1.056225,3628.0,2.428380,39.78,-121.86,1.178,1.178,NaN,NaN,NaN
6291,4.7589,16.0,5.200924,1.025404,4429.0,3.409546,34.01,-117.88,1.782,1.782,NaN,NaN,NaN
5444,3.3995,30.0,3.842576,1.067979,1341.0,2.398927,34.00,-118.43,3.240,3.240,NaN,NaN,NaN
3958,2.7540,21.0,2.642541,1.002954,2300.0,3.397341,34.20,-118.59,1.798,1.798,NaN,NaN,NaN
2754,1.4828,24.0,4.248322,0.922819,1083.0,3.634228,32.67,-115.49,0.731,0.731,NaN,NaN,NaN


Using classification algorithm to fill the missing Categorical values


In [15]:
import pandas as pd
import numpy as np # Import numpy and give it the alias 'np'

## applying logistic regression to find the missing values
missing_idx=[]
for i in range(df.shape[0]):
  if df['Road'][i] is None:
    missing_idx.append(i)
# Independent Parameters
missing_Road_X_train=np.array([ [df['MedInc'][i],df['AveBedrms'][i] ] for i in range(df.shape[0]) if i not in missing_idx ])
#Dependent  Parameter
missing_Road_Y_train=np.array([df['Road'][i] for i in range(df.shape[0]) if i not in missing_idx])
missing_Road_X_test = np.array([ [df['MedInc'][i],df['AveRooms'][i],df['AveBedrms'][i]] for i in range(df.shape[0]) if i in missing_idx])

In [16]:
import pandas as pd
import numpy as np

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split # Import train_test_split
# Code from ipython-input-12-980e55e5b82d to create df
loc_update = {"Country": [], "Road": [], "neighbourhood": []}
from sklearn.datasets import fetch_california_housing
housing= fetch_california_housing(as_frame=True)
df=housing.frame
df['target'] =housing.target

for i in loc_update.keys():
   df[i] = pd.Series(loc_update[i], index=df.index[:len(loc_update[i])]).reindex(df.index)

df = df.sample(axis=0, frac=1)
df.head(10)


## applying logistic regression to find the missing values
# Instead of using a placeholder, impute missing 'Road' values with a more meaningful strategy
# For example, you could use the most frequent value:

df = df.dropna(subset=['Road']) # Drop rows with missing 'Road' value  # Fixed indentation

# Check if there are at least two unique values in 'Road' after handling missing values
if len(df['Road'].unique()) < 2:
    print("Error: 'Road' column has less than two unique values. Cannot apply classification.")
else:
    # Now, proceed with creating your training and test data
    missing_idx=[]
    for i in range(df.shape[0]):
        if df['Road'][i] is None: # This condition is likely unnecessary now
            missing_idx.append(i)

    # Prepare your feature and target arrays
    X = np.array([ [df['MedInc'][i],df['AveBedrms'][i] ] for i in range(df.shape[0])])
    y = np.array([df['Road'][i] for i in range(df.shape[0])])

    # Split the data, stratifying based on 'Road' values where available
    missing_Road_X_train, missing_Road_X_test, missing_Road_Y_train, missing_Road_Y_test = train_test_split(
        X, # Features
        y, # Target variables
        test_size=0.2, # Correctly pass test_size as an argument
        stratify=y[~np.isin(np.arange(len(y)), missing_idx)]  # Stratify using non-missing 'Road' values
    )

    # model initialization
    model_1 =  SGDClassifier()

    # Fit the model regardless of the number of classes
    model_1.fit(missing_Road_X_train, missing_Road_Y_train)

    missing_Road_Y_Pred=model_1.predict(missing_Road_X_test)

    #model prediction
    missing_Road_Y_Pred= model_1.predict(missing_Road_X_test)
    print(missing_Road_Y_Pred)
    print(df['Road'].unique())



Error: 'Road' column has less than two unique values. Cannot apply classification.


In [17]:
# add the model back to the data frame

for n,i in enumerate( missing_idx):
  df["Road"][i] = missing_Road_Y_Pred[n]


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MedInc         0 non-null      float64
 1   HouseAge       0 non-null      float64
 2   AveRooms       0 non-null      float64
 3   AveBedrms      0 non-null      float64
 4   Population     0 non-null      float64
 5   AveOccup       0 non-null      float64
 6   Latitude       0 non-null      float64
 7   Longitude      0 non-null      float64
 8   MedHouseVal    0 non-null      float64
 9   target         0 non-null      float64
 10  Country        0 non-null      object 
 11  Road           0 non-null      object 
 12  neighbourhood  0 non-null      object 
dtypes: float64(10), object(3)
memory usage: 0.0+ bytes


In [19]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["Road"]= le.fit_transform(df["Road"])

PREDICTING MISSING COUNTY FEATURES

In [20]:
import pandas as pd
import numpy as np

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split # Import train_test_split
# Code from ipython-input-12-980e55e5b82d to create df
loc_update = {"Country": [], "Road": [], "neighbourhood": []}
from sklearn.datasets import fetch_california_housing
housing= fetch_california_housing(as_frame=True)
df=housing.frame
df['target'] =housing.target
# Add a 'county' column (replace with your actual county data)
# Example: Filling with a placeholder for demonstration
df['county'] = 'Placeholder'

for i in loc_update.keys():
   df[i] = pd.Series(loc_update[i], index=df.index[:len(loc_update[i])]).reindex(df.index)

df = df.sample(axis=0, frac=1)
df.head(10)

# Now you can proceed with dropping rows with missing 'county' values

df = df.dropna(subset=['county'])  # Drop rows with missing 'county' value  # Fixed indentation

# Check if there are at least two unique values in 'county' after handling missing values
if len(df['county'].unique()) < 2:
    print("Error: 'county' column has less than two unique values. Cannot apply classification.")
else:
    # Now, proceed with creating your training and test data
    missing_idx=[]
    for i in range(df.shape[0]):
        if df['county'][i] is None: # This condition is likely unnecessary now
            missing_idx.append(i)

    # Prepare your feature and target arrays
    X = np.array([ [df['MedInc'][i],df['AveBedrms'][i] ] for i in range(df.shape[0])])
    y = np.array([df['county'][i] for i in range(df.shape[0])])

    # Split the data, stratifying based on 'Road' values where available
    missing_Road_X_train, missing_Road_X_test, missing_Road_Y_train, missing_Road_Y_test = train_test_split(
        X, # Features
        y, # Target variables
        test_size=0.2, # Correctly pass test_size as an argument
        stratify=y[~np.isin(np.arange(len(y)), missing_idx)]  # Stratify using non-missing 'Road' values
    )
    # model initialization
    model_1 =  SGDClassifier()

    # Fit the model regardless of the number of classes
    model_1.fit(missing_Road_X_train, missing_Road_Y_train)

    missing_Road_Y_Pred=model_1.predict(missing_Road_X_test)

    #model prediction
    missing_Road_Y_Pred= model_1.predict(missing_Road_X_test)
    print(missing_Road_Y_Pred)
    print(df['county'].unique())

Error: 'county' column has less than two unique values. Cannot apply classification.


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20640 entries, 15089 to 20497
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MedInc         20640 non-null  float64
 1   HouseAge       20640 non-null  float64
 2   AveRooms       20640 non-null  float64
 3   AveBedrms      20640 non-null  float64
 4   Population     20640 non-null  float64
 5   AveOccup       20640 non-null  float64
 6   Latitude       20640 non-null  float64
 7   Longitude      20640 non-null  float64
 8   MedHouseVal    20640 non-null  float64
 9   target         20640 non-null  float64
 10  county         20640 non-null  object 
 11  Country        0 non-null      object 
 12  Road           0 non-null      object 
 13  neighbourhood  0 non-null      object 
dtypes: float64(10), object(4)
memory usage: 2.4+ MB


In [22]:
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal,target,county,Country,Road,neighbourhood
15089,3.5586,33.0,5.580882,0.985294,857.0,3.150735,32.80,-116.92,1.604,1.604,Placeholder,NaN,NaN,NaN
9772,3.0543,23.0,4.584245,1.172867,1858.0,4.065646,36.33,-121.23,0.924,0.924,Placeholder,NaN,NaN,NaN
14684,6.4335,7.0,7.383082,1.267241,4446.0,2.395474,32.83,-117.08,2.606,2.606,Placeholder,NaN,NaN,NaN
183,2.2500,50.0,4.207229,1.156627,1149.0,2.768675,37.80,-122.23,1.235,1.235,Placeholder,NaN,NaN,NaN
4378,3.9674,52.0,4.542254,1.074648,1559.0,2.195775,34.09,-118.27,2.688,2.688,Placeholder,NaN,NaN,NaN


UNDERSTANDING WHICH MODEL TO USE

In [23]:
# Dependent values
import pandas as pd  # Import pandas correctly
from sklearn.datasets import fetch_california_housing
# Fetch the California housing dataset from sklearn
housing = fetch_california_housing(as_frame=True)
df = housing.frame  # Get the DataFrame from the fetched dataset


Y = df.iloc[:,-3].values  # Use the existing df DataFrame
df_new = df.drop(labels= df.columns[-1], axis=1)  # Drop the last column by its index

X = df_new.iloc[:,:].values  # Use the new DataFrame

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size=0.33, random_state=42) # Use capital Y here to match your previous definition.


In [25]:
# Dependent values
Y = df.iloc[:,-3].values

# Drop the column used for Y (assuming it's the third from the end)
df = df.drop(df.columns[-3], axis=1 )  # Correctly assign the result back to df

X = df.iloc[:,:].values  # Now X will be based on the modified df

from sklearn.model_selection  import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
        X, Y, test_size=0.33, random_state=42)  # Use Y (capital Y) instead of y

from sklearn.ensemble  import  RandomForestRegressor
model = RandomForestRegressor()
model.fit(X_train,y_train)





RandomForestRegressor()

In [26]:
# model prediction
y_pred = model.predict(X_test)  # Use X_test (capital X)
# model accuracy
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)*100

97.41312535950699

add our own data


In [27]:
inp = np.array([6.0770, 34.0, 5.718841, 1.063768, 843.0, 2.443478, 33.88, -118.39])  # Remove the extra two values
inp = inp.reshape(1, -1)
model.predict(inp)

array([33.9448])